In [1]:
import numpy as np
import pandas as pd

# Data

In [2]:
data = pd.read_csv('./tvshows.csv')
data

,Title,Years,Certification,Runtime,Rating,Number of Votes,Emmys,Creators,Actors,Genres,Coutries of origins,Languages,Production companies,Link
0,Queen Cleopatra,2023,TV-14,45m,1.2,86K,0,NaN,"Jada Pinkett Smith, Adele James, Craig Russell...","Documentary, Drama, History",United Kingdom,English,Nutopia,https://www.imdb.com/title/tt27528139/?ref_=sr...
1,Velma,2023–2024,TV-MA,25m,1.6,80K,0,Charlie Grandy,"Mindy Kaling, Glenn Howerton, Sam Richardson, ...","Animation, Adventure, Comedy, Crime, Horror, M...","United States, South Korea",English,"Charlie Grandy Productions, Kaling Internation...",https://www.imdb.com/title/tt14153790/?ref_=sr...
2,Keeping Up with the Kardashians,2007–2021,TV-14,44m,2.9,32K,0,"Ryan Seacrest, Eliot Goldberg","Khloé Kardashian, Kim Kardashian, Kourtney Kar...","Family, Reality-TV",United States,"English, Spanish","Bunim-Murray Productions (BMP), Ryan Seacrest ...",https://www.imdb.com/title/tt1086761/?ref_=sr_...
3,Batwoman,2019–2022,TV-14,45m,3.6,47K,0,Caroline Dries,"Camrus Johnson, Rachel Skarsten, Meagan Tandy,...","Action, Adventure, Crime, Drama, Sci-Fi",United States,English,"Berlanti Productions, DC Entertainment, Warner...",https://www.imdb.com/title/tt8712204/?ref_=sr_...
4,The Acolyte,2024,TV-14,35m,4.1,125K,0,Leslye Headland,"Lee Jung-jae, Amandla Stenberg, Manny Jacinto,...","Action, Adventure, Drama, Fantasy, Mystery, Sc...",United States,English,"Lucasfilm, Disney+, The Walt Disney Company",https://www.imdb.com/title/tt12262202/?ref_=sr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,The Wire,2002–2008,TV-MA,1h,9.3,390K,0,David Simon,"Dominic West, Lance Reddick, Sonja Sohn, Wende...","Crime, Drama, Thriller",United States,"English, Greek, Mandarin, Spanish","Blown Deadline Productions, Home Box Office (HBO)",https://www.imdb.com/title/tt0306414/?ref_=sr_i_5
1134,Planet Earth,2006,TV-PG,50m,9.4,223K,4,NaN,"Sigourney Weaver, David Attenborough, Nikolay ...","Documentary, Family","United Kingdom, Canada, United States, Japan",English,"British Broadcasting Corporation (BBC), Canadi...",https://www.imdb.com/title/tt0795176/?ref_=sr_i_4
1135,Band of Brothers,2001,TV-MA,1h,9.4,544K,6,NaN,"Scott Grimes, Damian Lewis, Ron Livingston, Sh...","Drama, History, War","United Kingdom, United States","English, Dutch, French, German, Lithuanian","DreamWorks, DreamWorks Television, HBO Films",https://www.imdb.com/title/tt0185906/?ref_=sr_i_3
1136,Planet Earth II,2016,TV-G,50m,9.5,162K,2,NaN,"David Attenborough, Michael J. Sanderson, Gord...",Documentary,"United Kingdom, Germany, France, China, United...","English, French","BBC Natural History Unit (NHU), BBC America, Z...",https://www.imdb.com/title/tt5491994/?ref_=sr_i_2


# Check datatype of columns

In [3]:
data.dtypes

Title                    object
Years                    object
Certification            object
Runtime                  object
Rating                  float64
Number of Votes          object
Emmys                     int64
Creators                 object
Actors                   object
Genres                   object
Coutries of origins      object
Languages                object
Production companies     object
Link                     object
dtype: object

# Shape of data

In [4]:
#kích thước của dữ liệu
data.shape

(1138, 14)

# Check duplicate

In [5]:
data[data.duplicated(keep=False)]

,Title,Years,Certification,Runtime,Rating,Number of Votes,Emmys,Creators,Actors,Genres,Coutries of origins,Languages,Production companies,Link


Hoàn toàn không có 2 hàng nào hoàn toàn giống nhau

# Blank shell of each columns

In [6]:
data.isna().sum()

Title                     0
Years                     0
Certification            14
Runtime                  57
Rating                    0
Number of Votes           0
Emmys                     0
Creators                198
Actors                    1
Genres                    0
Coutries of origins       0
Languages                 0
Production companies     17
Link                      0
dtype: int64

# Change data in column Runtime about minute unit

In [7]:
new_col = []  
for val in data['Runtime']:
    if isinstance(val, str):
        if 'h' in val and 'm' in val: # Case have hour and minute
            hour, min = val.split('h')
            min = min.split('m')[0]
            new_col.append(int(hour) * 60 + int(min))
        elif 'h' in val:  # Case only hour
            hour = val.split('h')[0]
            new_col.append(int(hour) * 60)
        elif 'm' in val:  # Case only minute
            min = val.split('m')[0]
            new_col.append(int(min))
        elif pd.isna(val):
            new_col.append(0)
    else:
        new_col.append(val)  #if val is not str then append the list

data['Runtime'] = new_col
# Convert the 'Runtime' column to integer type, filling any remaining NaN with 0 first
data['Runtime'] = data['Runtime'].fillna(0).astype('int64')
print(data['Runtime'])

0       45
1       25
2       44
3       45
4       35
        ..
1133    60
1134    50
1135    60
1136    50
1137    45
Name: Runtime, Length: 1138, dtype: int64


# Change data in column Number of Votes about integer

In [8]:
new_col = []
for i in data['Number of Votes']:
    if 'K' in i:
        val = i.split('K')[0]
        new_col.append(int(float(val)*(10**3)))
    elif 'M' in i: 
        val = i.split('M')[0]
        new_col.append(int(float(val)*(10**6)))
data['Number of Votes'] = new_col
data['Number of Votes']

0         86000
1         80000
2         32000
3         47000
4        125000
         ...   
1133     390000
1134     223000
1135     544000
1136     162000
1137    2200000
Name: Number of Votes, Length: 1138, dtype: int64

# Handle column Actors
Fill Unknow into blank cell

In [9]:
new_col = []
for i in data['Actors']:
    if pd.isna(i):
        new_col.append('Unknown')
    else:
        new_col.append(i.split(','))
data['Actors'] = new_col


In [10]:
data['Actors']

0       [Jada Pinkett Smith,  Adele James,  Craig Russ...
1       [Mindy Kaling,  Glenn Howerton,  Sam Richardso...
2       [Khloé Kardashian,  Kim Kardashian,  Kourtney ...
3       [Camrus Johnson,  Rachel Skarsten,  Meagan Tan...
4       [Lee Jung-jae,  Amandla Stenberg,  Manny Jacin...
                              ...                        
1133    [Dominic West,  Lance Reddick,  Sonja Sohn,  W...
1134    [Sigourney Weaver,  David Attenborough,  Nikol...
1135    [Scott Grimes,  Damian Lewis,  Ron Livingston,...
1136    [David Attenborough,  Michael J. Sanderson,  G...
1137    [Bryan Cranston,  Aaron Paul,  Anna Gunn,  Bet...
Name: Actors, Length: 1138, dtype: object

# Handle data in column Years
- Character '–' diffirent '-' in keyboard
- Years will be converted to tuple: (year start, year end)
- If no year end then it will be None

In [11]:
new_col = []
for year in data['Years']:
    if '–' in year:
        first_year, second_year = year.split('–')
        first_year = int(first_year)
        if second_year == '':
            second_year = None
        else:
            second_year = int(second_year)
        # print(first_year, second_year)
        new_col.append((first_year, second_year))
new_col

[(2023, 2024),
 (2007, 2021),
 (2019, 2022),
 (2021, None),
 (2021, None),
 (2024, None),
 (2019, 2021),
 (1996, 2007),
 (2006, 2011),
 (1989, 2001),
 (2019, 2022),
 (2009, 2013),
 (2021, 2023),
 (2022, 2023),
 (2013, None),
 (2022, 2023),
 (2021, None),
 (2020, 2021),
 (2004, 2006),
 (2023, 2024),
 (2018, 2021),
 (2008, 2013),
 (2021, 2025),
 (2015, 2021),
 (2023, None),
 (2024, None),
 (2018, 2020),
 (2023, 2024),
 (2021, None),
 (2017, 2021),
 (2018, 2020),
 (1997, 2002),
 (2017, 2018),
 (2017, 2023),
 (2014, 2015),
 (1998, 2005),
 (1995, 1999),
 (2010, 2016),
 (2001, 2009),
 (2023, None),
 (2018, 2019),
 (2016, 2019),
 (2013, 2015),
 (2002, 2012),
 (2005, 2010),
 (2022, None),
 (2017, 2023),
 (1990, 2000),
 (2020, 2024),
 (2015, 2016),
 (2019, 2021),
 (2012, 2014),
 (2012, 2014),
 (2005, 2008),
 (1993, 1996),
 (1989, 1998),
 (2021, None),
 (2014, 2015),
 (2022, None),
 (2002, 2006),
 (2015, 2017),
 (2019, 2020),
 (2020, 2022),
 (2015, 2018),
 (1993, 1997),
 (2016, 2020),
 (2014, 20

# Handle column Genres

In [12]:
new_col = []
for i in data['Genres']:
    new_col.append(i.split(','))
data['Genres'] = new_col

# Handle column Countries of origin

In [13]:
new_col = []
for i in data['Coutries of origins']:
    new_col.append(i.split(','))
data['Coutries of origins'] = new_col

# Handle column Languages

In [14]:
new_col = []
for i in data['Languages']:
    new_col.append(i.split(','))
data['Languages'] = new_col

# Handle column Production companies
Fill Unknow into blank cell

In [15]:
new_col = []
for i in data['Production companies']:
    if pd.isna(i):
        new_col.append('Unknown')
    else:
        new_col.append(i.split(','))
data['Production companies'] = new_col

# Handle column Creators
Fill Unknow into blank cell

In [16]:
new_col = []
for i in data['Creators']:
    if pd.isna(i):
        new_col.append('Unknown')
    else:
        new_col.append(i.split(','))
data['Creators'] = new_col

# Explore Data

# Find show has max number of vote

In [17]:
max_vote = np.max(data['Number of Votes'])
print(max_vote)
highest_vote = []
for i, name in enumerate(data['Title']):
    if data['Number of Votes'][i] == max_vote:
        highest_vote.append(name)

print(highest_vote)


2400000
['Game of Thrones']


# Find show has max rating

In [18]:
max_rating = np.max(data['Rating'])
print(max_rating)
highest_rate = []
for i, name in enumerate(data['Title']):
    if data['Rating'][i] == max_rating:
        highest_rate.append(name)

print(highest_rate)

9.5
['Planet Earth II', 'Breaking Bad']


# Find the show has max emmys

In [21]:
max_emmys = np.max(data['Emmys'])
print(max_emmys)
highest_emmys = []
for i, name in enumerate(data['Title']):
    if data['Emmys'][i] == max_emmys:
        highest_emmys.append(name)
print(highest_emmys)

95
['Saturday Night Live']


# What is best show ?

In [19]:
df = pd.DataFrame(data)

# Min-Max Normalization
# For each column, apply the formula: (x - min) / (max - min)
df['Rating_norm'] = (df['Rating'] - df['Rating'].min()) / (df['Rating'].max() - df['Rating'].min())
df['Votes_norm'] = (df['Number of Votes'] - df['Number of Votes'].min()) / (df['Number of Votes'].max() - df['Number of Votes'].min())
df['Emmys_norm'] = (df['Emmys'] - df['Emmys'].min()) / (df['Emmys'].max() - df['Emmys'].min())


# Define weights for each factor
rating_weight = 1/3
votes_weight = 1/3
emmys_weight = 1/3

# Calculate the composite score using the weighted sum of normalized factors
df['Composite Score'] = (
    df['Rating_norm'] * rating_weight +
    df['Votes_norm'] * votes_weight +
    df['Emmys_norm'] * emmys_weight
)

# Select the show with the highest composite score
best_show = df.loc[df['Composite Score'].idxmax(), 'Title']
print(f"The best show is: {best_show}")

The best show is: Game of Thrones


# Find the show has long runtime 

## It helps us know to avoid long show, leads to boring

In [41]:
time = df[['Title', 'Runtime']]
time = time.sort_values(by='Runtime', ascending=False)
time.head(10)

,Title,Runtime
6,The Idol,276
124,Dune,265
1077,The Beatles: Get Back,150
972,Guardian: The Lonely and Great God,120
526,Resurrection: Ertugrul,120
805,Columbo,100
99,Salem's Lot,100
995,Poirot,100
1053,Das Boot,98
527,Hatfields &amp; McCoys,97
